This notebook requires pyfabm. For instructions on how to build and install pyfabm, see https://fabm.net/python

In [ ]:
# Note: "%matplotlib widget" below enables interactive plots but requires https://matplotlib.org/ipympl/
# Alternatively you could use "%matplotlib notebook" (interactive but deprecated) or "%matplotlib inline" (static plots)
%matplotlib widget
import matplotlib.pyplot
import scipy.integrate
import pyfabm

In [ ]:
# Load the model configuration from a YAML file
model = pyfabm.Model("../fabm-gotm-npzd.yaml")

In [ ]:
# Set environmental conditions and initialize the model
model.cell_thickness = 10.0  # cell thickness in m, used by getRates to scale surface and bottom fluxes
model.dependencies["surface_downwelling_photosynthetic_radiative_flux"].value = 70.0  # W/m2
model.dependencies["downwelling_photosynthetic_radiative_flux"].value = 10.0          # W/m2
assert model.start(), f"Model failed to start: {pyfabm.getError()}"

In [ ]:
# Time-integrate over 200 days (note: FABM's internal time unit is seconds!)
def dy(t, y):
    model.state[:] = y
    return model.getRates(t)


result = scipy.integrate.solve_ivp(
    dy, [0.0, 200.0 * 86400], model.state, first_step=3600.0, max_step=86400.0
)

In [ ]:
# Plot results
fig, ax = matplotlib.pyplot.subplots()
ax.plot(result.t / 86400, result.y.T)
ax.legend([v.long_name for v in model.state_variables])
ax.grid()
ax.set_xlabel("time (d)")
ax.set_ylabel(model.state_variables[0].units);